# 🐦 Extract Tweets from Data Science Club

This notebook uses **RapidAPI** to fetch the latest tweets from the Data Science Club account.

## Features:
- ✅ Free (50 requests/day)
- ✅ Reliable and working
- ✅ Easy to use

## Requirements:
1. Account on [RapidAPI](https://rapidapi.com/)
2. Subscription to [Twitter241 API](https://rapidapi.com/davethebeast/api/twitter241)

In [ ]:
# Install required libraries
!pip install requests

In [ ]:
import requests

# ==========================================
# 🔑 RapidAPI Key Setup
# ==========================================
# Get your Key from: https://rapidapi.com/davethebeast/api/twitter241

RAPIDAPI_KEY = "b7b8e17920msh995b55fdc65f61ap1f1417jsn33f3dd92a4b7"  # Put your API Key here

# Data Science Club username
DSC_USERNAME = "DSC_KAU"

In [ ]:
def get_tweets_rapidapi(username: str, rapidapi_key: str, count: int = 5):
    """
    🔑 Fetch tweets using RapidAPI (Free - 50 requests/day)
    """
    print(f"🔍 Fetching tweets for @{username} from RapidAPI...")
    print("=" * 50)
    
    user_url = "https://twitter241.p.rapidapi.com/user"
    tweets_url = "https://twitter241.p.rapidapi.com/user-tweets"
    
    headers = {
        "x-rapidapi-key": rapidapi_key,
        "x-rapidapi-host": "twitter241.p.rapidapi.com"
    }
    
    try:
        # Get user information
        print("🔄 Searching for account...")
        user_response = requests.get(
            user_url, 
            headers=headers, 
            params={"username": username}
        )
        
        if user_response.status_code == 403:
            print("❌ Invalid or expired API Key")
            return None
            
        if user_response.status_code != 200:
            print(f"❌ Error: {user_response.status_code}")
            return None
        
        user_data = user_response.json()
        
        # Extract user_id
        user_id = None
        if 'result' in user_data and 'data' in user_data['result']:
            user_id = user_data['result']['data']['user']['result']['rest_id']
        elif 'data' in user_data:
            user_id = user_data['data']['user']['result']['rest_id']
            
        if not user_id:
            print("❌ Account not found")
            print(f"Response: {user_data}")
            return None
            
        print(f"✅ Account found! User ID: {user_id}")
        
        # Fetch tweets
        print("🔄 Fetching tweets...")
        tweets_response = requests.get(
            tweets_url,
            headers=headers,
            params={"user": user_id, "count": str(count * 2)}
        )
        
        if tweets_response.status_code != 200:
            print(f"❌ Error fetching tweets: {tweets_response.status_code}")
            return None
        
        tweets_data = tweets_response.json()
        
        # Try to extract tweets using different methods
        tweets_list = []
        
        # Method 1: Old structure
        instructions = tweets_data.get('result', {}).get('timeline', {}).get('instructions', [])
        
        # Method 2: New structure
        if not instructions:
            instructions = tweets_data.get('data', {}).get('user', {}).get('result', {}).get('timeline_v2', {}).get('timeline', {}).get('instructions', [])
        
        # Search for entries
        entries = []
        for instruction in instructions:
            if 'entries' in instruction:
                entries = instruction['entries']
                break
            elif instruction.get('type') == 'TimelineAddEntries':
                entries = instruction.get('entries', [])
                break
        
        print(f"\n🐦 Last {count} tweets for @{username}:\n")
        
        tweet_count = 0
        for entry in entries:
            if tweet_count >= count:
                break
            
            entry_id = entry.get('entryId', '')
            
            # Skip non-tweets
            if 'tweet' not in entry_id and 'Tweet' not in entry_id:
                continue
                
            try:
                # Try to extract tweet
                content = entry.get('content', {})
                item_content = content.get('itemContent', {})
                tweet_results = item_content.get('tweet_results', {})
                result = tweet_results.get('result', {})
                
                # Handle TweetWithVisibilityResults
                if result.get('__typename') == 'TweetWithVisibilityResults':
                    result = result.get('tweet', {})
                
                legacy = result.get('legacy', {})
                
                if not legacy:
                    continue
                
                text = legacy.get('full_text', 'N/A')
                likes = legacy.get('favorite_count', 0)
                retweets = legacy.get('retweet_count', 0)
                created_at = legacy.get('created_at', 'N/A')
                
                tweet_data = {
                    'text': text,
                    'likes': likes,
                    'retweets': retweets,
                    'date': created_at
                }
                tweets_list.append(tweet_data)
                
                tweet_count += 1
                print(f"📝 Tweet {tweet_count}:")
                print(f"   📅 Date: {created_at}")
                print(f"   💬 Text: {text[:100]}..." if len(text) > 100 else f"   💬 Text: {text}")
                print(f"   ❤️ Likes: {likes}")
                print(f"   🔄 Retweets: {retweets}")
                print("-" * 40)
                
            except Exception as e:
                continue
        
        if not tweets_list:
            print("❌ No tweets found")
            print("\n📋 Debug - Response structure:")
            print(f"Keys: {tweets_data.keys()}")
            if 'result' in tweets_data:
                print(f"Result keys: {tweets_data['result'].keys() if isinstance(tweets_data['result'], dict) else 'N/A'}")
            
        return tweets_list
        
    except Exception as e:
        print(f"❌ Error occurred: {e}")
        return None

In [ ]:
# ==========================================
# 🚀 Run the code
# ==========================================

tweets = get_tweets_rapidapi(DSC_USERNAME, RAPIDAPI_KEY, count=5)

🔍 جاري جلب تغريدات @DSC_KAU من RapidAPI...
🔄 جاري البحث عن الحساب...
✅ تم العثور على الحساب! User ID: 1608405795707752448
🔄 جاري جلب التغريدات...

🐦 آخر 5 تغريدات لـ @DSC_KAU:

📝 التغريدة 1:
   📅 التاريخ: Tue Dec 02 18:49:33 +0000 2025
   💬 النص: يسر نادي علم البيانات الإعلان عن الفرق الفائزة في مسابقة أفضل مقترح✨

🥇 المركز الأول: بيانات واعية
🥈...
   ❤️ إعجابات: 23
   🔄 ريتويت: 2
----------------------------------------
📝 التغريدة 2:
   📅 التاريخ: Wed Nov 26 17:35:41 +0000 2025
   💬 النص: التخطيط📝! 
حلقة الوصل بين الفكرة والتنفيذ 🚀

نادي علم البيانات يقدم لكم ورشة بعنوان :
“التخطيط الفعّ...
   ❤️ إعجابات: 23
   🔄 ريتويت: 3
----------------------------------------
📝 التغريدة 3:
   📅 التاريخ: Wed Nov 26 15:16:46 +0000 2025
   💬 النص: جاهزين لمسابقة يقدمها نادي علم البيانات🔥؟

نعلن لكم عن فتح التسجيل في مسابقة:

“أفضل مقترح” 🎯✨

قدّم...
   ❤️ إعجابات: 40
   🔄 ريتويت: 8
----------------------------------------
📝 التغريدة 4:
   📅 التاريخ: Mon Nov 10 18:20:17 +0000 2025
   💬 النص: لأن العين

## 💾 Save Tweets to JSON File

In [ ]:
import json

def save_tweets_to_json(tweets, filename="dsc_tweets.json"):
    """Save tweets to a JSON file"""
    if not tweets:
        print("❌ No tweets to save")
        return
    
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(tweets, f, ensure_ascii=False, indent=2)
    
    print(f"✅ Saved {len(tweets)} tweets to: {filename}")

# Save the tweets
if tweets:
    save_tweets_to_json(tweets)

✅ تم حفظ 5 تغريدات في: dsc_tweets.json
